## Tópicos Especiais em Inteligência Artificial
### Classificação de diagnostico de cancer de mama com Deep Neural Network (DNN)
### Instituto Federal de Minas Gerais - IFMG
### Aluno: César Nogueira Rodrigues

Neste trabalho vamos utilizar o dataset Breast Cancer Wisconsin

1. ID do paciente 
2. Espessura (1-10)
3. Uniformidade de tamanho da célula (1-10)
3. Uniformidade de forma da célula (1-10)
5. Adesão marginal (1-10)
6. Tamanho único epitelial da célula (1-10)
7. Quantidade de núcleos nus (1-10)
8. Cromatina (1-10)
9. Quantidade de nucleos normais (1-10)
10. Mitoses (1-10)
11. [SAIDA] Classificação (2 para indicar BENIGNO, 4 Para indicar MALIGNO)

O dataset contém 458 amostras com: 
- 65.5% de casos BENIGNO
- 34.5% de casos MALIGNO 

In [1]:
# função para deixar o jupyter com celulas preenchendo a tela toda

from IPython.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))

In [7]:
#importação de bibliotecas 

# exportacao do requiriment.txt
import subprocess
import sys 

#  lib para dados e graficos
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from tabulate import tabulate


ModuleNotFoundError: No module named 'tabulate'